In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import io
import re
import zipfile

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from src.data_preprocessing import preprocessing
# import fiftyone as fo
# import fiftyone.zoo as foz

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "bigbucket"  # @param {type:"string"}
from datetime import datetime
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse
!mkdir -p mush
!mkdir -p inat
bucket_name='medium_mush'
bucket_name2='inatdatabase'
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name} mush
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name2} inat

YOLOv5 🚀 v6.1-201-g9a7f289 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.3/166.8 GB disk)
Project ID:  
deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  40919      0 --:--:-- --:--:-- --:--:-- 40919
OK
Hit:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-secur

In [ ]:
%cd /content/mush/yolov5
#git clone https://github.com/ultralytics/yolov5  # clone
%pip install -qr requirements.txt  # install

import torch
import utils

In [ ]:
!python train.py --img 416 --batch 16 --epochs 30 --data /content/yolov5/mush/Mushrooms.yaml --cfg ./models/yolov5s.yaml --weights  /content/yolov5/mush/yolo/ --name yolov5s_results  --cache --device 0

train: weights=/content/yolov5/mush/yolo/, cfg=./models/yolov5s.yaml, data=/content/yolov5/mush/Mushrooms.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-201-g9a7f289 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

In [ ]:
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/morel/inat --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name morel_inat
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/inat/MO --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name morel_MO
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/gyro/inat --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name gyro_inat
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/gyro/MO --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name gyro_MO
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/ascos --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name ascos
!python detect.py --save-crop --weights /content/mush/yolov5/runs/train/yolov5s_results/weights/best.pt --source /content/inat/MO/basidios --conf 0.25 --exist-ok --project /content/mush/fungi/images/inat --name basidios